In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
#Imports
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [45]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/titanic kaggle/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/titanic kaggle/test.csv")
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


For now variables selected have been limited to those of which are continuous or trivial to convert to a continuous format, i.e sex. This is because sklearn models only accept continous variables as input. Embarked is also fairly trivial to convert using one-hot encoding.

In [46]:
cont_vars = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']

In [47]:
df_train = df_train[cont_vars]

In [10]:
train1, test1 = train_test_split(df_train, test_size=0.3, shuffle=False)

In [11]:
train1.isna().sum()

Pclass        0
Sex           0
Age         131
SibSp         0
Parch         0
Fare          0
Survived      0
dtype: int64

Missing values found in the age column shall be resolved by replacing these values with the average age. Other options exist such as ffill or bfill which would fill in the values based on the nearest existing value in either direction. Another method could be to approximate age based upon the other factors present.

In [12]:
train1['Age'].fillna(train1['Age'].mean(), inplace=True)
train1.replace(['male','female'], [0,1], inplace=True)

In [13]:
test1['Age'].fillna(test1['Age'].mean(), inplace=True)
test1.replace(['male','female'], [0,1], inplace=True)

#Train test split for clf training


In [14]:
train_X = train1[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
train_Y = train1['Survived']

In [15]:
test_X = test1[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
test_Y = test1['Survived']

#Testing Various Classifiers and measuring accuracy

In [19]:
#Support Vector Machine Classifier
clf_svc = SVC()
clf_svc.fit(train_X, train_Y)
preds_svc = clf_svc.predict(test_X) 
print("SVC Classifier Accuracy: {}".format(accuracy_score(test_Y, preds_svc)))

SVC Classifier Accuracy: 0.7014925373134329


In [20]:
#XGBoost
clf_xg = XGBClassifier()
clf_xg.fit(train_X, train_Y)
preds_xg = clf_xg.predict(test_X)
print("XGBoost Classifier Accuracy: {}".format(accuracy_score(test_Y, preds_xg)))

XGBoost Classifier Accuracy: 0.8432835820895522


In [21]:
#Logistic Regression
clf_LR = LogisticRegression()
clf_LR.fit(train_X, train_Y)
preds_LR = clf_LR.predict(test_X)
print("Logistic Regression Classifier Accuracy: {}".format(accuracy_score(test_Y, preds_LR)))

Logistic Regression Classifier Accuracy: 0.7947761194029851


In [22]:
#K-Nearest Neighbours Classifier
clf_KN = KNeighborsClassifier()
clf_KN.fit(train_X, train_Y)
preds_KN = clf_KN.predict(test_X)
print("K-Nearest Neigbours Classifier Accuracy: {}".format(accuracy_score(test_Y, preds_KN)))

K-Nearest Neigbours Classifier Accuracy: 0.7238805970149254


In [23]:
#Multinomial Naive Bayes Classifier
clf_NB = MultinomialNB()
clf_NB.fit(train_X, train_Y)
preds_NB = clf_NB.predict(test_X)
print("Multinomial Naive Bayes Classifier Accuracy: {}".format(accuracy_score(test_Y, preds_NB)))

Multinomial Naive Bayes Classifier Accuracy: 0.7276119402985075


#Choosing XGBoost as our final model and running this model on our actual test data after training on the entirety of the training set.

In [48]:
df_test_X = df_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]

In [49]:
final_train_X = df_train.loc[:, df_train.columns != "Survived"]
final_train_Y = df_train['Survived']

final_train_X['Age'].fillna(final_train_X['Age'].mean(), inplace=True)
final_train_X.replace(['male','female'], [0,1], inplace=True)

df_test_X['Age'].fillna(df_test_X['Age'].mean(), inplace=True)
df_test_X.replace(['male','female'], [0,1], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [50]:
clf_xg.fit(final_train_X, final_train_Y)
final_pred = clf_xg.predict(df_test_X)

In [51]:
df_test['Survived'] = final_pred
out = df_test[['PassengerId','Survived']]

In [52]:
out

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


Below saves the output for submission on kaggle as a zip file

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
out.to_csv('out.zip', index=False,
          compression=compression_opts)  